<a href="https://colab.research.google.com/github/yunyoungwoo/2024S-Ajou-ML-FP/blob/main/temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import json
import pandas as pd

# Colab 환경에 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import json
import pandas as pd

# DataFrame을 만들기 위한 빈 리스트 생성
image_paths = []
labels = []

# training 및 validation 디렉토리 경로
training_directory = '/content/drive/MyDrive/Colab Notebooks/dataset/training'
validation_directory = '/content/drive/MyDrive/Colab Notebooks/dataset/training'

# 클래스 이름 리스트 생성
class_names = os.listdir(training_directory)

# 필요한 정보만 추출하여 리스트에 추가
for class_name in class_names:
    # training 데이터 처리
    training_image_directory = os.path.join(training_directory, class_name)
    training_label_file = os.path.join(training_image_directory, f'label{class_name}.json')

    # JSON 파일 불러오기
    with open(training_label_file, 'r') as f:
        training_data = json.load(f)

    # JSON 파일에서 필요한 정보 추출하여 리스트에 추가
    for image_data in training_data:
        image_id = image_data['image_id']
        image_file_name = image_data['image_file_name']
        value_6 = image_data['value_6']

        # 이미지 경로
        image_path = os.path.join(training_image_directory, f'image{image_id}.jpg')

        image_paths.append(image_path)
        labels.append(value_6)

# DataFrame 생성
df = pd.DataFrame({'image_path': image_paths, 'value_6': labels})

# DataFrame 확인
print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/path_to_json_file.json'

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 원-핫 인코딩
encoder = OneHotEncoder()
labels_one_hot = encoder.fit_transform(df['label'].values.reshape(-1, 1)).toarray()

# DataFrame에 원-핫 인코딩된 라벨 추가
df_encoded = df.copy()
df_encoded[['label_0', 'label_1', 'label_2', 'label_3']] = labels_one_hot

# 데이터 증강 설정
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

# 이미지 경로와 라벨 가져오기
image_paths = df_encoded['image_path'].tolist()
labels = df_encoded[['label_0', 'label_1', 'label_2', 'label_3']].values

# 데이터 증강 적용 후 이미지와 라벨 생성
augmented_images = []
augmented_labels = []

for image_path, label in zip(image_paths, labels):
    image = plt.imread(image_path)
    image = image.reshape((1,) + image.shape)  # 모델에 맞는 shape로 변환
    label = label.reshape((1,) + label.shape)  # 모델에 맞는 shape로 변환

    # 데이터 증강 수행
    for x_batch, y_batch in datagen.flow(image, label, batch_size=1):
        augmented_images.append(x_batch[0])
        augmented_labels.append(y_batch[0])
        break  # 한 번 증강된 데이터만 사용

# DataFrame에 증강된 이미지 경로와 라벨 추가
df_augmented = pd.DataFrame({'image_path': augmented_images,
                             'label_0': [label[0] for label in augmented_labels],
                             'label_1': [label[1] for label in augmented_labels],
                             'label_2': [label[2] for label in augmented_labels],
                             'label_3': [label[3] for label in augmented_labels]})

# 증강된 DataFrame 확인
print(df_augmented.head())



In [1]:
import numpy as np
import pandas as pd
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [1]:
import random
import torch

seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
import os
from PIL import Image
from tqdm import tqdm
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
train_path = "/kaggle/input/2022-ai-tp-18011886/train"
test_path = "/kaggle/input/2022-ai-tp-18011886/test"
all_label = os.listdir(train_path)

In [ ]:
class train_dataset():
    def __init__(self):
        self.data_path = []
        self.label_idx = []

        for idx, label in enumerate(all_label):
            for data_path in tqdm(os.listdir(os.path.join(train_path,label))):
                path = os.path.join(label,data_path)
                self.data_path.append(path)
                self.label_idx.append(idx)

    def __getitem__(self,idx):
        transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((320,320)),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

        path = self.data_path[idx]
        img = Image.open(os.path.join(train_path,path))
        img = img.convert('RGB')
        img = transform(img)
        return img, self.label_idx[idx]

    def __len__(self):
        return len(self.data_path)

class test_dataset():
    def __init__(self):
        self.data_path = []

        for data_path in tqdm(os.listdir(os.path.join(test_path))):
            path = data_path
            self.data_path.append(path)

        self.data_path.sort()

    def __getitem__(self,idx):
        transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((320,320)),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

        path = self.data_path[idx]
        img = Image.open(os.path.join(test_path,path))
        img = img.convert('RGB')
        img = transform(img)
        return img, 0

    def __len__(self):
        return len(self.data_path)

In [ ]:
batch_size = 32
train_dataset = train_dataset()
test_dataset = test_dataset()
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                         shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size,
                         shuffle=False)

In [ ]:
import torchvision.models as models

model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(2048, 4, bias=True)
torch.nn.init.xavier_normal_(model.fc.weight)
model.to(device)

In [ ]:
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [ ]:
model.train()

epochs = 5

for epoch in range(epochs):
    avg_cost = 0

    for x,y in tqdm(train_loader):
        #import pdb;pdb.set_trace()
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        out = model(x).to(device)
        cost = loss(out, y).to(device)
        cost.backward()
        optimizer.step()
        scheduler.step()

        avg_cost +=cost.item()/batch_size

    print(epoch+1,avg_cost)

In [ ]:
with torch.no_grad():
    model.eval()
    corr_pred = list()
    test = list()
    for x,_ in test_loader:
        x = x.to(device)
        out = model(x).to(device)
        pred = torch.argmax(out,dim=1).cpu().detach().numpy()
        corr_pred.extend(pred)

    for i in range(len(corr_pred)):
        test.append(all_label[corr_pred[i]])

In [ ]:
submit = pd.read_csv('/kaggle/input/2022-ai-tp-18011886/sample_submit.csv',encoding='cp949')
submit['label'] = test
submit.to_csv('submit.csv',index=False,encoding='cp949')